In [ ]:
!pip install groq langchain langchain-core langchain-groq chromadb pypdf gradio sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 817.7/817.7 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 299.3/299.3 kB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 526.8/526.8 kB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.2/12.2 MB 27.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 21.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.4/116.4 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 24.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from langchain_groq import ChatGroq
from langchain_community.document_loaders import TextLoader, PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
# from langchain_community import embeddings
from langchain_core.prompts import ChatPromptTemplate
# from langchain_core.runnables import RunnablePassthrough
# from langchain_core.output_parsers import StrOutputParser
from google.colab import userdata
import os
import time
import textwrap
import gradio as gr

In [ ]:
loader = PyPDFDirectoryLoader("/content/drive/MyDrive/Ds")
text = loader.load()

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = text_splitter.split_documents(text)

In [ ]:
chunk_index = 0  # You can change this index to view different chunks
print("\nChunk", chunk_index + 1, "of", len(chunks), ":")
print(chunks[chunk_index])


Chunk 1 of 62 :
page_content='1The Guide to \nModern Employee \nOnboarding\nBEST PRACTICES, CHECKLISTS \nAND TEMPLATES FOR HR & \nPEOPLE TEAMS' metadata={'source': '/content/drive/MyDrive/Ds/guide.pdf', 'page': 0}


In [ ]:
from transformers import AutoModel
from langchain.embeddings import HuggingFaceEmbeddings

model_name = "sentence-transformers/all-mpnet-base-v2"
model_kwargs = {"device": "cpu"}

embeddings_hf = HuggingFaceEmbeddings(model_name=model_name, model_kwargs=model_kwargs)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
vectorstore = Chroma.from_documents(
    documents = chunks,
    collection_name= "groq_embeds",
    embedding = embeddings_hf,
)

retriever = vectorstore.as_retriever()

In [ ]:
from google.colab import userdata
import os
from langchain_groq import ChatGroq

os.environ["GROQ_API_KEY"] = userdata.get('groq_api_key')
llm = ChatGroq(temperature=0, model_name = "mixtral-8x7b-32768" )

In [ ]:
from langchain.chains import RetrievalQA

rag_template = """
Welcome to our HR Assistant! I'm here to help make your onboarding experience smooth and informative.

Here's what you need to know:

{context}

---

Now, let's tackle your question: {question}

If your query is not directly covered by the information provided, I will use my general HR knowledge to help you out. Rest assured, all responses are based on up-to-date and reliable HR practices.
"""

rag_prompt = ChatPromptTemplate.from_template(rag_template)
qa_chain = RetrievalQA.from_chain_type(
    llm, retriever=vectorstore.as_retriever(), chain_type_kwargs={"prompt": rag_prompt},
)

In [ ]:
response = qa_chain.invoke("Explain the pdf")

In [ ]:
print(response)

{'query': 'Explain the pdf', 'result': 'Thank you for the detailed information on the onboarding process. It seems like you\'ve covered a lot of ground!\n\nRegarding your question about the PDF, I assume you are referring to the PDF file containing the "14 Superannuation standard choice form (NAT 13080)."\n\nThis form is used in Australia for employees to choose their superannuation fund, which is a retirement savings plan. Superannuation is a mandatory contribution made by employers towards their employees\' retirement savings. In cases where an employee does not make a choice, the employer can designate a default fund, which is the "employer nominated fund" mentioned in the onboarding information.\n\nThe NAT 13080 form is a standard choice form provided by the Australian government for employees to select their preferred superannuation fund. Employers are required to provide this form to their employees and follow the instructions outlined in the form to ensure compliance with supera

In [ ]:
response['result']

'Thank you for the detailed information on the onboarding process. It seems like you\'ve covered a lot of ground!\n\nRegarding your question about the PDF, I assume you are referring to the PDF file containing the "14 Superannuation standard choice form (NAT 13080)."\n\nThis form is used in Australia for employees to choose their superannuation fund, which is a retirement savings plan. Superannuation is a mandatory contribution made by employers towards their employees\' retirement savings. In cases where an employee does not make a choice, the employer can designate a default fund, which is the "employer nominated fund" mentioned in the onboarding information.\n\nThe NAT 13080 form is a standard choice form provided by the Australian government for employees to select their preferred superannuation fund. Employers are required to provide this form to their employees and follow the instructions outlined in the form to ensure compliance with superannuation guarantee requirements.\n\nBy 

In [ ]:
def process_question(user_question):
  response = qa_chain.invoke(user_question)
  full_response = response['result']
  return full_response

In [ ]:
def process_question(user_question):
    response = qa_chain.invoke(user_question)
    full_response = response['result']

    # Extracting source/metadata information
    source_metadata = response.get('metadata', {}).get('sources', 'Source information not available')

    # Combining response text with source/metadata information
    response_with_metadata = f"Response: {full_response}\n\nSource/Metadata: {source_metadata}"

    return response_with_metadata

In [ ]:
interface = gr.Interface(fn= process_question,
                         inputs= gr.Textbox(lines=2, placeholder="Type your question here"),
                         outputs= gr.Textbox(),
                         title= "HRbot",
                         description="Ask any question about onboarding process")

In [ ]:
interface.launch()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://78532a648c5ba14ac2.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
